<a href="https://colab.research.google.com/github/NahedElshareif/dilivary-order/blob/master/bert_to_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Dependencies

In [1]:
%%capture
!pip install git-python==1.0.3
!pip install sacrebleu==1.4.2
!pip install rouge_score
!pip install farasapy
!git clone https://github.com/aub-mind/arabert
!pip install pyarabic
!pip install datasets
!pip install -U transformers==4.5.1
!git clone https://github.com/aub-mind/Arabic-Empathetic-Chatbot

# New Section

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Getting the dataset from Github

In [3]:
!wget https://raw.githubusercontent.com/aub-mind/Arabic-Empathetic-Chatbot/master/arabic-empathetic-conversations.csv

--2021-07-13 10:40:38--  https://raw.githubusercontent.com/aub-mind/Arabic-Empathetic-Chatbot/master/arabic-empathetic-conversations.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7873052 (7.5M) [text/plain]
Saving to: ‘arabic-empathetic-conversations.csv’

arabic-empathetic-c 100%[===================>]   7.51M  --.-KB/s    in 0.07s   

2021-07-13 10:40:38 (108 MB/s) - ‘arabic-empathetic-conversations.csv’ saved [7873052/7873052]



Importing Necessary Libraries

In [4]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset 
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from sacrebleu import corpus_bleu

Specifying the encoder/decoder maximum sequence length and the BERT model name

In [5]:
encoder_max_length=150
decoder_max_length=150
model_name = "aubmindlab/bert-base-arabert"

Specificying the AraBERT tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

[link text](https:// [link text](https://))Loading the dataset using ArabicEmpatheticDialogues.py

In [7]:
all_data = load_dataset("/content/Arabic-Empathetic-Chatbot/models/ArabicEmpatheticDialogues.py")
train_data = all_data['train'].train_test_split(test_size=0.1,seed=42)['train']
val_data = all_data['train'].train_test_split(test_size=0.1,seed=42)['test']
dev_data = val_data.train_test_split(test_size=0.5,seed=42)['train']
test_data = val_data.train_test_split(test_size=0.5,seed=42)['test']

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


 99%|█████████▉| 239M/241M [00:25<00:00, 9.89MiB/s]

[2021-07-13 10:41:18,520 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


100%|██████████| 241M/241M [00:40<00:00, 9.89MiB/s]

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/arabic_emp_conv/arabic_emp_conv/1.0.0/af81e5e61abf371a71d8d27b4483bf0141f8b51c942075e3fed5df7b73769b6d/cache-4c167943656b84bf.arrow and /root/.cache/huggingface/datasets/arabic_emp_conv/arabic_emp_conv/1.0.0/af81e5e61abf371a71d8d27b4483bf0141f8b51c942075e3fed5df7b73769b6d/cache-727e21c709706d35.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/arabic_emp_conv/arabic_emp_conv/1.0.0/af81e5e61abf371a71d8d27b4483bf0141f8b51c942075e3fed5df7b73769b6d/cache-1af606f3f1154fcf.arrow and /root/.cache/huggingface/datasets/arabic_emp_conv/arabic_emp_conv/1.0.0/af81e5e61abf371a71d8d27b4483bf0141f8b51c942075e3fed5df7b73769b6d/cache-891e6c1cae7f81d7.arrow


Dataset arabic_emp_conv downloaded and prepared to /root/.cache/huggingface/datasets/arabic_emp_conv/arabic_emp_conv/1.0.0/af81e5e61abf371a71d8d27b4483bf0141f8b51c942075e3fed5df7b73769b6d. Subsequent calls will reuse this data.


In [8]:
#Lengths of train/dev/test sets
print("Length of train data",len(train_data))
print("Length of dev data",len(dev_data))
print("Length of test data",len(test_data))

Length of train data 32965
Length of dev data 1831
Length of test data 1832


Define function to process data in model input format

In [9]:
def process_data_to_model_inputs(batch):                                                               
    # Tokenizer will automatically set [BOS] <text> [EOS]                                               
    inputs = tokenizer(batch["context"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["response"], padding="max_length", truncation=True, max_length=decoder_max_length)
                                                                                                        
    batch["input_ids"] = inputs.input_ids                                                               
    batch["attention_mask"] = inputs.attention_mask                                                     
    batch["decoder_input_ids"] = outputs.input_ids                                                      
    batch["labels"] = outputs.input_ids.copy()                                                          
    # mask loss for padding                                                                             
    batch["labels"] = [                                                                                 
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]                     
    batch["decoder_attention_mask"] = outputs.attention_mask                                                                              
                                                                                                         
    return batch  

Specify Batch Size

In [10]:
batch_size=16

Prepare the train/dev/test sets

In [11]:
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["context", "response"],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

dev_data = dev_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["context", "response"],
)
dev_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

test_data = test_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["context", "response"],
)
test_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Create the BERT2BERT architecture initialized with AraBERT pre-trained checkpoints

In [12]:
arabert2arabert = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name, tie_encoder_decoder=False)

Some weights of the model checkpoint at aubmindlab/bert-base-arabert were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabert and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias

Specifying parameters

In [13]:
#set special tokens
arabert2arabert.config.decoder_start_token_id = tokenizer.cls_token_id                                             
arabert2arabert.config.eos_token_id = tokenizer.sep_token_id
arabert2arabert.config.pad_token_id = tokenizer.pad_token_id

#sensible parameters for beam search
#set decoding params                               
arabert2arabert.config.max_length = 64
arabert2arabert.config.early_stopping = True

arabert2arabert.config.num_beams = 1
arabert2arabert.config.vocab_size = arabert2arabert.config.encoder.vocab_size

Specifying Metrics

In [14]:
import torch
import torch.nn as nn

def compute_metrics(pred):
  labels_ids = pred.label_ids
  #pred_ids = torch.argmax(pred.predictions,dim=2)
  pred_ids = pred.predictions  

  # all unnecessary tokens are removed
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  return {"bleu": round(corpus_bleu(pred_str , [label_str]).score, 4)}

Specifying Training Arguments (Not tuned)

In [15]:
#Set training arguments 
training_args = Seq2SeqTrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,
    gradient_accumulation_steps = 2,
    predict_with_generate=True,
    do_eval=True,
    evaluation_strategy ="epoch",
    do_train=True,
    logging_steps=500,  
    save_steps= 32965 // ( batch_size * 2),  
    warmup_steps=100,
    eval_steps=10,
    #max_steps=16, # delete for full training
    num_train_epochs=3,# uncomment for full training
    overwrite_output_dir=True,
    save_total_limit=0,
    #fp16=True, 
)

Instantiate the trainer

In [16]:
trainer = Seq2SeqTrainer(
    model=arabert2arabert,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=dev_data,
    tokenizer=tokenizer
)

Train the BERT2BERT model

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Runtime,Samples Per Second
0,3.368000,3.098559,0.743600,239.083100,7.658000
1,2.957500,2.908542,0.858700,213.802600,8.564000
2,2.724900,2.854956,1.335200,235.691400,7.769000


TrainOutput(global_step=3090, training_loss=3.1987823930758874, metrics={'train_runtime': 9176.3255, 'train_samples_per_second': 0.337, 'total_flos': 5.3190392454144e+16, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2311921664, 'init_mem_gpu_alloc_delta': 1197381632, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -1019740160, 'train_mem_gpu_alloc_delta': 3609379328, 'train_mem_cpu_peaked_delta': 1037869056, 'train_mem_gpu_peaked_delta': 8652998656})

Evaluate the BERT2BERT model

In [18]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_bleu': 1.3352,
 'eval_loss': 2.8549563884735107,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 930568192,
 'eval_runtime': 235.6054,
 'eval_samples_per_second': 7.771}

Save & Reload the BERT2BERT model

In [19]:
trainer._save("./arabert2arabert")
tokenizer.save_pretrained("./arabert2arabert")

('./arabert2arabert/tokenizer_config.json',
 './arabert2arabert/special_tokens_map.json',
 './arabert2arabert/vocab.txt',
 './arabert2arabert/added_tokens.json')

## Chat with the model

In [20]:
#Import pipeline
from transformers import pipeline

chatbot = pipeline("text2text-generation",model='./arabert2arabert')

In [21]:
chatbot("انا مريض جدا اليوم",
    num_beams=10,
    max_length=200,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


[{'generated_text': 'أوه لا! أنا آسف لسماع ذلك. هل انت بخير ؟'}]

In [22]:
chatbot("أنا متحمس حقًا لعطلة نهاية الأسبوع هذه",
    num_beams=10,
    max_length=200,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3)

[{'generated_text': 'ما هي خططك المفضلة ؟'}]

In [23]:
chatbot("لقد كنت دائمًا مخلصًا لصديقي ، وأنا فخور بذلك.",
    num_beams=10,
    max_length=200,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3)

[{'generated_text': 'أنا متأكد من أنك ستكون بخير!'}]

In [24]:
chatbot("كان الطعام جيدا حقًا. لكني شعرت بتعب شديد بعد الاكل.",
    num_beams=10,
    max_length=200,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3)

[{'generated_text': 'هذا يبدو وكأنه الكثير من المرح.'}]

In [25]:
chatbot("أنا حقا أريد أن أفقد الوزن. اعلم انني استطيع فعلها",
    num_beams=10,
    max_length=200,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3)

[{'generated_text': 'أنا متأكد من أنك ستبلي بلاء حسنا!'}]

In [31]:
chatbot("كم عمرك؟",
    num_beams=10,
    max_length=200,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3)

[{'generated_text': 'أوه ، واو! أنا آسف لسماع ذلك.'}]